In [1]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

from nltk.tokenize import word_tokenize 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import eli5

In [2]:
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding='utf8', sep='\t')

In [3]:
df['sentiment'].value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [4]:
# так как задача бинарной классификации, уберём класс neutral
df=df[df['sentiment'] != 'neautral'] 

In [5]:
# посмотрим на пример отзыва
df.iloc[0]['review']
# нужна предобработка текста: убрать знаки препинания и привести к одному регистру

'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

In [6]:
# убрали знаки препинания (регулярное выражение заменяет знаки на пустой символ)
# чтобы ничего случайно не затереть, запишем в отдельную колонку review_processed
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values

In [7]:
# привели к нижнему регистру
df['review_processed'] = df['review_processed'].str.lower()

In [8]:
# токенизация 
df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))

100%|██████████| 60000/60000 [00:12<00:00, 4745.23it/s]


In [9]:
# результат
df['review_processed'].iloc[0]

['качество',
 'плохое',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не',
 'соответствует',
 'ткань',
 'ужасная',
 'рисунок',
 'блеклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'ужас',
 'не',
 'стоит',
 'за',
 'такие',
 'деньги',
 'г']

In [10]:
morph = pymorphy2.MorphAnalyzer()

In [11]:
# приводим к нормальной форме
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])

100%|██████████| 60000/60000 [03:57<00:00, 253.15it/s]


In [12]:
# результат
df['review_lemmatized'].iloc[0]

['качество',
 'плохой',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не',
 'соответствовать',
 'ткань',
 'ужасный',
 'рисунок',
 'блёклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'ужас',
 'не',
 'стоить',
 'за',
 'такой',
 'деньга',
 'г']

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [14]:
# векторизация
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))

In [15]:
X.shape

(60000, 51549)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

In [18]:
logit = LogisticRegression()

In [19]:
logit.fit(X_train, y_train)

LogisticRegression()

In [21]:
y_pred = logit.predict_proba(X_test)[:, 1]
y_pred

array([0.18457169, 0.6284437 , 0.99264536, ..., 0.25177362, 0.00206928,
       0.00343717])

In [22]:
roc_auc_score(y_test, y_pred)

0.9735222897024753